In [ ]:
import os
import sys

# intrusense 폴더를 sys.path에 추가
sys.path.append(os.path.abspath('../'))

from dotenv import load_dotenv
import scripts.db_functions as dbf
# .env 파일에서 환경변수 로드
load_dotenv()

# 정의된 칼럼 리스트
columns = [
    'Destination Port', 'Flow Duration', 'Total Fwd Packets',
       'Total Backward Packets', 'Total Length of Fwd Packets',
       'Total Length of Bwd Packets', 'Fwd Packet Length Max',
       'Fwd Packet Length Min', 'Fwd Packet Length Mean',
       'Fwd Packet Length Std', 'Bwd Packet Length Max',
       'Bwd Packet Length Min', 'Bwd Packet Length Mean',
       'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s',
       'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min',
       'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max',
       'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std',
       'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags',
       'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Min Packet Length', 'Max Packet Length', 'Packet Length Mean',
       'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count',
       'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count',
       'URG Flag Count', 'CWE Flag Count', 'ECE Flag Count', 'Down/Up Ratio',
       'Average Packet Size', 'Avg Fwd Segment Size', 'Avg Bwd Segment Size',
       'Fwd Header Length.1', 'Fwd Avg Bytes/Bulk', 'Fwd Avg Packets/Bulk',
       'Fwd Avg Bulk Rate', 'Bwd Avg Bytes/Bulk', 'Bwd Avg Packets/Bulk',
       'Bwd Avg Bulk Rate', 'Subflow Fwd Packets', 'Subflow Fwd Bytes',
       'Subflow Bwd Packets', 'Subflow Bwd Bytes', 'Init_Win_bytes_forward',
       'Init_Win_bytes_backward', 'act_data_pkt_fwd', 'min_seg_size_forward',
       'Active Mean', 'Active Std', 'Active Max', 'Active Min', 'Idle Mean',
       'Idle Std', 'Idle Max', 'Idle Min', 'Label'
       ]

# 라벨 인코딩
label_mapping = {
    'BENIGN': 0, 'FTP-Patator': 1, 'SSH-Patator': 2, 'DoS slowloris': 3,
    'DoS Slowhttptest': 4, 'DoS Hulk': 5, 'DoS GoldenEye': 6, 'Heartbleed': 7,
    'Web Attack � Brute Force': 8, 'Web Attack � XSS': 9,
    'Web Attack � Sql Injection': 10, 'Infiltration': 11, 'Bot': 12,
    'PortScan': 13, 'DDoS': 14
}


# MySQL 연결 정보
user = os.getenv('DB_USER')
password = os.getenv('DB_PASSWORD')
host = os.getenv('DB_HOST')
port = int(os.getenv('DB_PORT'))
database = os.getenv('DB_DATABASE')

# 0.5s

### CSV 파일들을 DB로 저장

In [ ]:
# 테이블 이름
table_name = 'network_data_table'
label_table_name = 'label_mapping_table'
source_table_name = 'source_file_table'

# 파일 선택
data_files = dbf.select_files()

# DB 엔진 생성
engine = dbf.create_db_engine(user, password, host, port, database)

# 테이블 처리 및 데이터 저장
dbf.check_and_drop_table(engine, table_name, columns)
dbf.save_label_mapping(engine, label_mapping, label_table_name)
dbf.process_and_save_csv_to_db(engine, data_files, columns, table_name, source_table_name, label_mapping) 
# 칼럼명의 좌우 공백 제거 후 저장
# 결측치 처리: Imputation
#     - 'Flow Duration'이 0인 경우 Flow Bytes/s, Flow Packets/s가 NaN 혹은 무한으로 0으로 대체하여 저장

# DB 연결 종료
engine.dispose()

# 7m 20.9s

'network_data_table' 테이블이 삭제되었습니다.
'network_data_table' 테이블이 생성되었습니다.
라벨 매핑 정보가 'label_mapping_table' 테이블에 저장되었습니다.
C:/projects/Intrusense/data/raw/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv 데이터베이스에 저장 완료
C:/projects/Intrusense/data/raw/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv 데이터베이스에 저장 완료
C:/projects/Intrusense/data/raw/Friday-WorkingHours-Morning.pcap_ISCX.csv 데이터베이스에 저장 완료
C:/projects/Intrusense/data/raw/Monday-WorkingHours.pcap_ISCX.csv 데이터베이스에 저장 완료
C:/projects/Intrusense/data/raw/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv 데이터베이스에 저장 완료
C:/projects/Intrusense/data/raw/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv 데이터베이스에 저장 완료
C:/projects/Intrusense/data/raw/Tuesday-WorkingHours.pcap_ISCX.csv 데이터베이스에 저장 완료
C:/projects/Intrusense/data/raw/Wednesday-workingHours.pcap_ISCX.csv 데이터베이스에 저장 완료
파일 정보가 'source_file_table' 테이블에 저장되었습니다.


### Query 실행하여 DB 저장

In [ ]:
# DB 엔진 생성
engine = dbf.create_db_engine(user, password, host, port, database)

# 테이블 이름
table_name = 'network_data_table'

# 저장할 CSV 파일 경로
output_csv_path = os.path.join(os.getcwd(), f'../data/processed/db_{table_name}.csv')

# 실행할 쿼리
query = f"SELECT * FROM {table_name}"

# 쿼리 실행 결과를 CSV로 저장
dbf.run_sql_and_save_to_csv(engine, query, output_csv_path)

# DB 연결 종료
engine.dispose()

# 3m 7.0s

SQL 실행 결과가 'c:\projects\Intrusense\notebooks\../data/processed/db_network_data_table.csv'에 저장되었습니다.


### DB의 모든 테이블 이름 출력

In [ ]:
# DB 엔진 생성
engine = dbf.create_db_engine(user, password, host, port, database)

# 모든 테이블 목록
tables = dbf.get_tables(engine)

# 테이블 이름 출력
print(f"{database}에 존재하는 테이블:")
for i, table in enumerate(tables, 1):
    print(f"{i}. {table}")

# DB 연결 종료
engine.dispose()

# 0.0s

intrusense에 존재하는 테이블:
1. label_mapping_table
2. network_data_table
3. source_file_table


### 각 테이블의 데이터 확인 (최대 20개)

In [ ]:
# 각 테이블의 데이터 확인 (최대 20개)
for table in tables:
    print(f"{table}")
    print(dbf.get_table_data(engine, table))
    print("================")

# 0.0s

label_mapping_table
                         Label  Encoded Value
0                       BENIGN              0
1                  FTP-Patator              1
2                  SSH-Patator              2
3                DoS slowloris              3
4             DoS Slowhttptest              4
5                     DoS Hulk              5
6                DoS GoldenEye              6
7                   Heartbleed              7
8     Web Attack � Brute Force              8
9             Web Attack � XSS              9
10  Web Attack � Sql Injection             10
11                Infiltration             11
12                         Bot             12
13                    PortScan             13
14                        DDoS             14
network_data_table
    id  Destination Port  Flow Duration  Total Fwd Packets  \
0    1           54865.0            3.0                2.0   
1    2           55054.0          109.0                1.0   
2    3           55055.0           52.0